# 목표

#### 1. 기본적으로 주가를 예측하는 것을 목표로 한다.
#### 2. 주가를 예측하기 위해, 주가 데이터만 사용하는 것이 아니라 기업의 재무상태 또한 고려하려 한다.
#### 3. 여기서 문제점은 주가의 종가는 매일 업데이트 되지만 재무상태는 1년에 한 번 공시된다는 것이다. 
#### 4. 물론 분기별 재무보고서가 있지만 수정되는 경우도 많고, 정확하지 않은 경우가 많아 사업계획서를 기준으로 한다.

## Imports

#### 데이터 분석 라이브러리

In [ ]:
import pandas as pd
import numpy as np

#### 시각화 라이브러리

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#### 머신러닝용 라이브러리

In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score

#### 기타 필요한 라이브러리

In [4]:
import warnings
warnings.filterwarnings('ignore')
from fqtoolkit import dgToDf as dtd
import statsmodels.api as sm
import riskfolio as rp

## Data 불러오기

O 데이터는 기본적으로 데이터 가이드에서 재공되는 주가정보와 재무 정보를 사용한다.    
   
- 기업 : 코스피 시총 상위 50개 기업 + 코스닥 시총 상위 50개 기업 
- 기간 : 2020.01 ~ 2022.05  
- 기준 : 월말, 1~12월 data same
- 단위 : KRW(원)


O 재무 정보는 재무재표를 기준으로 하고, Feature로 **Asset, Liability, Sale, Income, Price**를 사용한다.

- assets : 총자산 
- liability : 총부채
- sale : 매출액
- income : 영업이익
- price : 수정주가(배당반영)

In [5]:
assets, liability, sale, income, price = [item[1] for item in dtd("data2.xlsx", "Sheet1", 5).items()]

- 문제는 앞서 말한 주가 정보와 재무재표의 기간이 맞지 않는다는 것인데, 주가 데이터는 당월 종가를 기준으로 하고, 재무제표는 당년 6월에 발표된다고 가정하였다. 
- 2020년 1월에 투자를 한다고 가정하면, 2019년 6월에 발표된 재무제표를 기준으로 판단할 수 밖에 없기 때문에 price를 shift 해줘 주가 데이터와 재무 정보의 시작점을 같게 한다.

In [6]:
assets0 = assets.pct_change()
liability0 = liability.pct_change()

In [7]:
price.shift(12);

In [8]:
price = price.dropna(axis=1)
returns = price.shift(12+6).pct_change()

returns

,삼성전자,SK하이닉스,삼성SDI,현대차,카카오,기아,POSCO홀딩스,현대모비스,한국전력,SK텔레콤,...,기업은행,롯데케미칼,한국조선해양,한온시스템,HLB,CJ ENM,동진쎄미켐,동화기업,네이처셀,주성엔지니어링
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2000-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31,0.096598,-0.027025,0.093534,0.294771,0.284103,0.260543,0.112072,0.070313,-0.020492,0.045026,...,0.003782,0.008978,0.020595,0.099343,-0.097196,0.043517,0.122576,0.483533,0.048673,0.248111
2022-02-28,-0.067365,-0.092993,0.138365,0.395263,0.184868,0.050754,-0.046515,0.090025,0.073279,0.124718,...,-0.002512,0.124630,-0.033632,0.295209,0.038276,0.031274,0.050619,-0.092836,0.123418,-0.064293
2022-03-31,0.084328,0.118508,-0.041989,0.011329,-0.104420,0.104813,0.070547,0.026787,-0.004848,-0.040314,...,-0.009931,0.036941,-0.040603,-0.037232,0.194703,0.198822,-0.106929,-0.083426,-0.075117,0.064243


### Clustering

In [12]:
X = pd.DataFrame()
for i in range(0,len(assets.index),12):
    X = X.append(assets.iloc[i])
X

,삼성전자,LG에너지솔루션,SK하이닉스,삼성바이오로직스,NAVER,삼성SDI,현대차,카카오,LG화학,기아,...,네이처셀,주성엔지니어링,SFA반도체,덕산네오룩스,메가스터디교육,NICE평가정보,피엔티,케어젠,와이지엔터테인먼트,에이비엘바이오
2000-01-31,4.642150e+10,NaN,2.128287e+10,NaN,3.640202e+07,5.663565e+09,3.198721e+10,9.197135e+07,NaN,8.828701e+09,...,20534862.00,2.008390e+08,2.200540e+07,NaN,NaN,5.533616e+07,NaN,NaN,NaN,NaN
2001-01-31,5.211488e+10,NaN,1.441334e+10,NaN,4.600652e+07,5.630804e+09,3.987230e+10,1.450934e+08,3.636326e+09,9.812333e+09,...,16418794.00,2.138113e+08,3.623972e+07,NaN,NaN,7.368140e+07,NaN,NaN,NaN,NaN
2002-01-31,6.495499e+10,NaN,1.115963e+10,NaN,1.252079e+08,5.725090e+09,4.594547e+10,1.625884e+08,4.618144e+09,1.081520e+10,...,15552957.00,1.401451e+08,4.208461e+07,NaN,NaN,7.760807e+07,NaN,NaN,NaN,NaN
2003-01-31,6.804174e+10,NaN,8.603848e+09,NaN,2.006867e+08,6.409798e+09,5.342682e+10,2.121888e+08,5.835172e+09,1.351332e+10,...,8762283.00,1.179045e+08,7.808548e+07,NaN,NaN,8.831396e+07,NaN,NaN,NaN,NaN
2004-01-31,6.900462e+10,NaN,8.846875e+09,NaN,3.713483e+08,6.722372e+09,5.802313e+10,4.051957e+08,6.897243e+09,1.439987e+10,...,7919411.00,1.603942e+08,9.728073e+07,NaN,NaN,8.811036e+07,NaN,1.268583e+06,NaN,NaN
2005-01-31,7.446180e+10,NaN,1.069729e+10,NaN,4.055770e+08,6.700613e+09,6.607877e+10,4.353898e+08,7.291334e+09,1.603449e+10,...,9562844.00,1.863002e+08,1.498995e+08,NaN,NaN,1.022882e+08,NaN,3.352043e+06,NaN,NaN
2006-01-31,8.136621e+10,NaN,1.538491e+10,NaN,5.822588e+08,6.888625e+09,7.070948e+10,3.467310e+08,7.638435e+09,1.734294e+10,...,8779267.00,2.065712e+08,1.653741e+08,NaN,NaN,1.049335e+08,8.474074e+06,3.826036e+06,NaN,NaN
2007-01-31,9.337514e+10,NaN,1.771830e+10,NaN,8.561845e+08,7.114692e+09,8.384753e+10,2.208749e+08,8.550754e+09,1.946100e+10,...,6456017.00,2.744838e+08,3.766747e+08,NaN,NaN,1.165803e+08,1.129816e+07,5.878208e+06,1.091431e+07,NaN
2008-01-31,1.053006e+11,NaN,1.657622e+10,NaN,1.219211e+09,6.768821e+09,1.032058e+11,2.973824e+08,9.735573e+09,2.558355e+10,...,40045643.00,3.087595e+08,3.889821e+08,NaN,NaN,1.317103e+08,1.945529e+07,7.487283e+06,1.739275e+07,NaN
2009-01-31,1.121798e+11,NaN,1.630353e+10,NaN,1.605435e+09,7.147735e+09,1.023249e+11,3.078796e+08,1.053059e+10,2.596288e+10,...,28582445.00,3.675428e+08,4.138146e+08,NaN,NaN,NaN,2.695206e+07,1.433815e+07,2.027773e+07,NaN


In [13]:
ax,an = plot_dendrogram(returns=X, codependence='spearman',
                        linkage='ward', k=None, max_k=5,
                        leaf_order=True, ax=None)
ax;

NameError: name 'plot_dendrogram' is not defined

In [14]:
color = list(set(an['leaves_color_list']))
new_df = pd.DataFrame()
new_df['CODE'] = an['leaves']
new_df['SORTED'] = an['leaves_color_list']
new_df = new_df.sort_values(by=['CODE'])
new_df['COMPANY'] = X.columns
li = []
for i in color:
    li.append(new_df['COMPANY'][new_df['SORTED']==i])

len(li[3])

NameError: name 'an' is not defined

### Rolling
- Rolling 기간은 122달로 설정하였다.

In [ ]:
rolling_period = 122
results = {}

for col in returns.columns:
    reSeries = {}
    for n in range(1, returns.shape[0]-rolling_period):
        
        temp = pd.DataFrame({"asset":assets0[col].iloc[n:n+rolling_period-1],
                            "liability":liability0[col].iloc[n:n+rolling_period-1],
                            "sale":sale[col].iloc[n:n+rolling_period-1],
                            "income":income[col].iloc[n:n+rolling_period-1],
                             "re0": returns[col].iloc[n-1:n+rolling_period-2],
                            "re": returns[col].iloc[n:n+rolling_period-1]})
        print(temp)
        temp = temp.dropna()
        future = returns[col].iloc[n+120]
        
        if temp.shape[0] < 50:
            continue
            
        model = DecisionTreeRegressor(max_depth=6)#
        model.fit(temp.drop("re", axis=1).iloc[:-1,:], temp["re"].iloc[:-1])
        
        r = model.predict(temp.drop("re", axis=1).iloc[[-1], :])
        
        reSeries[temp.index[-1]] = temp["re"].iloc[-1]-r[0]
        
    if len(reSeries) > 0:
        results[col] = pd.Series(reSeries)

In [ ]:
list(li[0])

In [ ]:
r2 = []
acc = []

for n in range(12, 48):
    results = pd.DataFrame(results)
    signal = (results).mean(axis=1)

    signal = (signal-signal.shift(1).rolling(n).mean())/signal.shift(1).rolling(n).std() #변경 말것!
    temp = pd.DataFrame({"returns":returns.shift(-1).mean(axis=1), "result":signal}).dropna()

    test = temp.iloc[int(temp.shape[0]/4):2*int(temp.shape[0]/4), :]
    test0 = temp.iloc[2*int(temp.shape[0]/4):, :]
    
    training = temp.iloc[:int(temp.shape[0]/4), :]

    mu = training.result.mean()
    sd = training.result.std()

    test.result = (test.result-mu)/sd
    test0.result = (test0.result-mu)/sd
    training.result = (training.result - mu) /sd

    test["returns+"] = np.where(test["returns"] > 0, test["returns"], 0)
    test0["returns+"] = np.where(test0["returns"] > 0, test0["returns"], 0)

    test["port"] = test["returns"] * np.where(-test.result>2, 1, 0)
    test0["port"] = test0["returns"] *np.where(-test0.result>2, 1, 0)
    
    acc.append(accuracy_score(np.where(test.returns>0, 1, 0), np.where(-test.result>0, 1, 0)))

    from sklearn.metrics import accuracy_score

    print(f"{np.corrcoef(test.returns, -test.result)[1,0 ] ** 2 * 100:.4f}%")
    r2.append(np.corrcoef(test.returns, -test.result)[1,0 ] ** 2)

In [ ]:
pd.Series(r2, index=range(12, 48)).argmax()+12

In [ ]:
plt.plot(range(12, 48), r2)

In [ ]:
n = 33

results = pd.DataFrame(results)

signal = (results).mean(axis=1)
#
signal = (signal-signal.shift(1).rolling(n).mean())/signal.shift(1).rolling(n).std() #변경 말것!
temp = pd.DataFrame({"returns":returns.shift(-1).mean(axis=1), "result":signal}).dropna()
test = temp.iloc[int(temp.shape[0]/4):2*int(temp.shape[0]/4), :]
test0 = temp.iloc[2*int(temp.shape[0]/4):, :]

mu = temp.result.iloc[:int(temp.shape[0]/4)].mean()
sd = temp.result.iloc[:int(temp.shape[0]/4)].std()

test.result = (test.result-mu)/sd

test0.result = (test0.result-mu)/sd


test["returns+"] = np.where(test["returns"] > 0, test["returns"], 0)
test0["returns+"] = np.where(test0["returns"] > 0, test0["returns"], 0)

test["port"] = test["returns"] * -1*test.result#np.where(-1*test.result >0,, 0)
test0["port"] = test0["returns"] * np.where( -1*test0.result > 0, 1, 0)

from sklearn.metrics import classification_report

print(f"{np.corrcoef(test.returns, -test.result)[1,0 ] ** 2 * 100:.4f}%")
    
print(classification_report(np.where(test.returns> 0, 1, 0), np.where(-test.result >0, 1, 0)))

In [ ]:
print(f"{np.corrcoef(test0.returns, -test0.result)[1,0 ] ** 2 * 100:.4f}%")
    
print(classification_report(np.where(test0.returns> 0, 1, 0), np.where(-test0.result >0, 1, 0)))

In [ ]:
plt.plot(test0.result,test0.returns, "o")

In [ ]:
plt.plot((1+test0["port"]*1).cumprod())
plt.plot((1+test0.returns*1).cumprod())

In [ ]:
plt.plot(test0.returns ,test0["port"], "o")

In [ ]:
print(test0["port"].mean() * np.sqrt(12)/ test0["port"].std())
print(test0["returns"].mean() * np.sqrt(12)/ test0["returns"].std())

In [ ]:
test0.corr()

In [10]:
sm.OLS(test0["port"], sm.add_constant(test0[["returns", "returns+"]])).fit().summary()

NameError: name 'test0' is not defined

In [11]:
plt.plot((1+test0["returns"]).cumprod())

NameError: name 'test0' is not defined

In [22]:
np.mean(np.where(test0["returns"]>0, 1, 0))

0.6491228070175439

In [23]:
from itertools import combinations as c

def estimation(bench, port):
    n0 = 0
    n = 0
    for t1, t2, t3 in c(range(len(bench)), 3):
        n0 += 1 
        rms = bench.iloc[[t1, t2, t3]].sort_values() 
        trueT1, trueT2, trueT3 = rms.index 
        betaH = (port.loc[trueT3] - port.loc[trueT2]) / (bench.loc[trueT3] - bench.loc[trueT2]) 
        betaL = (port.loc[trueT2] - port.loc[trueT1]) / (bench.loc[trueT2] - bench.loc[trueT1])
        
        if betaH > betaL:       
            n += 1          
    return 2*(n / n0) -1
          
def kernel(rms, ris):
    
    rm1, rm2, rm3 = rms
    ri1, ri2, ri3 = ris   
    premise = rm1 < rm2 and rm2 < rm3
    
    if not premise:       
        return 0
    
    betaH = (ri3 - ri2) / (rm3 - rm2)        
    betaL = (ri2 - ri1) / (rm2 - rm1)
    conclusion = betaH > betaL
       
    if  premise and not conclusion:       
        return 0   
    else:
      
        return 1

def stanardE(bench, port, esti):
    sum0 = 0
    for t1 in bench.index:   
        sum1 = 0     
        n0 = 0    
        for t2, t3 in c(bench.index, 2):            
            n0 += 1           
            sum1 += kernel(bench.loc[[t1, t2, t3]], port.loc[[t1, t2, t3]])
        sum1 = (sum1 / n0 - esti) ** 2       
        sum0 += sum1
    var = 9 / len(bench.index) * sum0
    return np.sqrt(var)
    
from scipy.stats import norm

def nonParaMT(bench, port):
    
    esti = estimation(bench, port)
    se = stanardE(bench, port, esti)
    statistic = esti * np.sqrt(len(bench)) /se
    pVal = 1 - norm.cdf(statistic, loc=0, scale=1)
    return esti, se , statistic, pVal

In [24]:
nonParaMT(test0["returns"], test0["port"])

(-0.25037593984962403,
 0.9963193242547572,
 -1.8972801655271554,
 0.9711045143312425)